Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from math import sqrt

In [2]:
movies_df = pd.read_csv(r'C:\Users\Administrator\Downloads\movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies_df.shape

(34208, 3)

The movie data set has 34208 rows and 3 columns

In [4]:
#removing year from title to decrease confusion with movies having years in their name
#storing the year in another column making the shape of the dataset to (34208,4)
#using regular expression to find the year after the movie name
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))', expand = False)
movies_df['year'] = movies_df.year.str.extract('(\(\d\d\d\d\))', expand = False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,(1995)
1,2,Jumanji,Adventure|Children|Fantasy,(1995)
2,3,Grumpier Old Men,Comedy|Romance,(1995)
3,4,Waiting to Exhale,Comedy|Drama|Romance,(1995)
4,5,Father of the Bride Part II,Comedy,(1995)
...,...,...,...,...
34203,151697,Grand Slam,Thriller,(1967)
34204,151701,Bloodmoney,(no genres listed),(2010)
34205,151703,The Butterfly Circus,Drama,(2009)
34206,151709,Zero,Drama|Sci-Fi,(2015)


In [5]:
#making the genre column easy to read
movies_df['genres'] = movies_df['genres'].str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",(1995)
1,2,Jumanji,"[Adventure, Children, Fantasy]",(1995)
2,3,Grumpier Old Men,"[Comedy, Romance]",(1995)
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",(1995)
4,5,Father of the Bride Part II,[Comedy],(1995)


# Using one hot encoding method to convert the genres from list to vectors
This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. 

In [6]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#now,
for index, row in movies_df.iterrows():
    for genres in row['genres']:
        moviesWithGenres_df.at[index, genres]=1
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",(1995),1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",(1995),1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",(1995),0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",(1995),0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],(1995),0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
ratings_df = pd.read_csv(r'C:\Users\Administrator\Downloads\ratings.csv')
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [8]:
#timestamp column not needed
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


# Content-Based recommendation system
This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects.

In [9]:
#USER INPUT
user_input = [
                {'title': 'Iron Man', 'rating': '5'},
                {'title': 'Captain America: The First Avenger', 'rating': '4.5'},
                {'title': 'Now You See Me', 'rating': '3.5'},
                {'title': 'National Treasure', 'rating': '4'},
                {'title': 'Harry Potter and the Order of the Phoenix', 'rating': '5'},
                {'title': 'Avengers: Age of Ultron', 'rating': '4.5'}
        ]
inputMovies = pd.DataFrame(user_input)
inputMovies

,title,rating
0,Iron Man,5
1,Captain America: The First Avenger,4.5
2,Now You See Me,3.5
3,National Treasure,4
4,Harry Potter and the Order of the Phoenix,5
5,Avengers: Age of Ultron,4.5


 Add movieid to input user

In [11]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,8972,National Treasure,4
1,54001,Harry Potter and the Order of the Phoenix,5
2,59315,Iron Man,5
3,88140,Captain America: The First Avenger,4.5
4,102903,Now You See Me,3.5
5,122892,Avengers: Age of Ultron,4.5


In [12]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
8290,8972,National Treasure,"[Action, Adventure, Drama, Mystery, Thriller]",(2004),1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11986,54001,Harry Potter and the Order of the Phoenix,"[Adventure, Drama, Fantasy, IMAX]",(2007),1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12646,59315,Iron Man,"[Action, Adventure, Sci-Fi]",(2008),1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17544,88140,Captain America: The First Avenger,"[Action, Adventure, Sci-Fi, Thriller, War]",(2011),1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
21167,102903,Now You See Me,"[Crime, Mystery, Thriller]",(2013),0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26471,122892,Avengers: Age of Ultron,"[Action, Adventure, Sci-Fi]",(2015),1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
inputMovies['rating'] = inputMovies['rating'].astype('float64', errors = 'ignore')
inputMovies['rating']

0    4.0
1    5.0
2    5.0
3    4.5
4    3.5
5    4.5
Name: rating, dtype: float64

In [15]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             23.0
Animation              0.0
Children               0.0
Comedy                 0.0
Fantasy                5.0
Romance                0.0
Drama                  9.0
Action                18.0
Crime                  3.5
Thriller              12.0
Horror                 0.0
Mystery                7.5
Sci-Fi                14.0
IMAX                   5.0
Documentary            0.0
War                    4.5
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Using this, we can recommend movies that satisfy the user's preferences.

In [16]:
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
genreTable

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,,
1,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",(1995),1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Jumanji,"[Adventure, Children, Fantasy]",(1995),1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Grumpier Old Men,"[Comedy, Romance]",(1995),0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Waiting to Exhale,"[Comedy, Drama, Romance]",(1995),0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,Father of the Bride Part II,[Comedy],(1995),0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151697,151697,Grand Slam,[Thriller],(1967),0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151701,151701,Bloodmoney,[(no genres listed)],(2010),0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
151703,151703,The Butterfly Circus,[Drama],(2009),0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
genreTable.shape
#new shape after dropping

(34208, 20)

In [19]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.275862
2    0.275862
3    0.000000
4    0.088670
5    0.000000
dtype: float64

In [20]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
115479    0.827586
71999     0.798030
91500     0.748768
8361      0.748768
58025     0.748768
dtype: float64

# The Final Recommendation Table (Top 20 movies)

In [21]:
#The final recommendation table
Final_table=movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

In [22]:
Final_table.set_index(Final_table['movieId'])
Final_table.drop('movieId', 1)

,title,genres,year
4686,Megiddo: The Omega Code 2,"[Action, Adventure, Fantasy, Sci-Fi, Thriller]",(2001)
4923,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",(1991)
6261,"Matrix Reloaded, The","[Action, Adventure, Sci-Fi, Thriller, IMAX]",(2003)
6823,"Matrix Revolutions, The","[Action, Adventure, Sci-Fi, Thriller, IMAX]",(2003)
7763,"Day After Tomorrow, The","[Action, Adventure, Drama, Sci-Fi, Thriller]",(2004)
9403,"Sound of Thunder, A","[Action, Adventure, Drama, Sci-Fi, Thriller]",(2005)
10382,"Cave, The","[Action, Adventure, Horror, Mystery, Sci-Fi, T...",(2005)
11410,Children of Men,"[Action, Adventure, Drama, Sci-Fi, Thriller]",(2006)
11838,Spider-Man 3,"[Action, Adventure, Sci-Fi, Thriller, IMAX]",(2007)
12464,Jumper,"[Action, Adventure, Drama, Sci-Fi, Thriller]",(2008)
